<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2022notebooks/2022_0422first_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ファイルをアップロードします
# [HAD 公式ダウンロードページ](https://osf.io/32cyp/) から直接ダウンロードする方法が分かりません
# なので，一旦，自身の PC へダウンロードしてから，このセルを実行してください
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install --upgrade xlrd
    !pip install --upgrade pandas
    from google.colab import files
    #files?
    files.upload()  # ご自身の PC からファイルをアップロードして下さい

In [2]:
import os
import sklearn.datasets
import pandas as pd
import numpy as np

#X, y = sklearn.datasets.load_iris(return_X_y=True,as_frame=True)
#_X, _y = X.to_numpy(), y.to_numpy()

had_dir = '.'  if isColab else '/Users/asakawa/study/2022HAD'
had_samples = pd.read_excel(os.path.join(had_dir, 'HAD_sample_data.xls'),sheet_name='iris')
X = had_samples[['Sepal.L','Sepal.W', 'Petal.L', 'Petal.W']].to_numpy()
# sepal:萼片, petal:花弁
# `花は内側から外に向かって、雌しべ、雄しべ、花弁、萼片、そして場合によっては苞という順番で器官が並んでいます。`
# `ですので、雄しべのすぐ外にある器官が花弁で、そのさらに外にある器官が萼片ということになります。`
# https://jspp.org/hiroba/q_and_a/detail.html?id=1219 より
y = had_samples[['Species']].to_numpy()
_y = np.zeros((y.shape[0],3))
for i, __y in enumerate(y):
     _y[i,__y[0]-1] = 1
        
y = np.copy(_y)        
#y.shape[1]


In [ ]:
import numpy as np
np.set_printoptions(precision=2)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

X = torch.Tensor(X)
y = torch.Tensor(y)
lr = 0.01

class MLP(nn.Module):
    
    def __init__(self, n_inp, n_hid, n_out=1):
        super().__init__()
        self.n_inp = n_inp
        self.n_hid = n_hid
        self.n_out = n_out
        self.hid_layer = nn.Linear(in_features =self.n_inp, 
                                   out_features=self.n_hid,
                                   bias=True)
        self.out_layer = nn.Linear(self.n_hid, self.n_out)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.hid_layer(x)
        x = self.sigmoid(x)
        x = self.out_layer(x)
        x = self.sigmoid(x)
        return x
    
mlp = MLP(n_inp=X.shape[1], n_hid=8, n_out=y.shape[1])
#loss_f = nn.MSELoss()
#loss_f = nn.CrossEntropyLoss()
loss_f = nn.BCELoss()
#optim_f = torch.optim.SGD(mlp.parameters(),lr=lr)
optim_f = torch.optim.Adam(mlp.parameters(), lr=lr)

mlp.eval()
_y = mlp(X)
_pre_loss = loss_f(_y, y)
print(f'訓練開始前の損失値:    {_pre_loss.item():.3f}')

mlp.train()
epochs = 500
for epoch in range(epochs):
    optim_f.zero_grad()
    
    _y = mlp(X)                  # モデルに処理させて出力を得る
    loss = loss_f(_y, y)         # 損失値の計算

    if epoch % (epochs>>2) == 0: # 途中結果の出力
        print(f'エポック:{epoch:4d}: 損失値:{loss.item():.3f}')

    loss.backward()              # 誤差逆伝播
    optim_f.step()               # 重み更新。すなわち学習


In [ ]:
mlp.eval()
_y = mlp(X)
for i,z in enumerate(_y):
    print(f'{i:2d} {z.detach().numpy()}')     # 結果の出力

In [ ]:
import numpy as np                                # 必要となるライブラリの輸入
np.set_printoptions(precision=3)

had_dir = '.'  if isColab else '/Users/asakawa/study/2022HAD'
had_samples = pd.read_excel(os.path.join(had_dir, 'HAD_sample_data.xls'),sheet_name='iris')
X = had_samples[['Sepal.L','Sepal.W', 'Petal.L', 'Petal.W']].to_numpy()
y = had_samples[['Species']].to_numpy()
_y = np.zeros((y.shape[0],3))
for i, y1 in enumerate(y):
     _y[i,y1[0]-1] = 1

y = np.copy(_y)        

lr, N_hid = 0.002, 8                              # lr: 学習率, N_hid: 中間層数
Wh = np.random.random((X.shape[1], N_hid)) - 1/2  # 入力層から中間層への結合係数の初期化
Wo = np.random.random((N_hid, y.shape[1])) - 1/2  # 中間層から出力層への結合係数の初期化
epochs = 500
for t in range(epochs):                           # 繰り返し
    H  = np.tanh(np.dot(X, Wh))                   # 入力層から中間層への計算。ハイパータンジェント関数
    _y = 1/(1. + np.exp(-(np.dot(H, Wo))))        # 中間層から出力層への計算。シグモイド関数
    Dy = (y - _y) * (_y * (1. - _y))              # 誤差の微分
    DH = Dy.dot(Wo.T) * (1. - H ** 2)             # 誤差逆伝播
    Wo += lr * H.T @ Dy
    Wh += lr * X.T @ DH                           # 中間層から入力層への重み更新
    if t % (epochs>>2) == 0:
        print(np.array((y-_y)**2).mean())

for i,z in enumerate(_y):
    print(f'{i:2d} {z}')                          # 結果の出力